In [1]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import torchvision
import torch
from torch import nn

%matplotlib inline
import matplotlib.pyplot as plt

from pathlib import Path

import pickle
from sklearn.model_selection import train_test_split
from scipy.special import softmax

In [173]:
b = 4
t = 11
first_blow = 6
embd_size = 13

idx = torch.rand(b, t)  # 4 x 1023

In [178]:
def apply_batchwise(func, M):
    tList = [ func(m) for m in torch.unbind(M, dim=0) ]  # batch is first index
    return torch.stack(tList, dim=0)


def minmax_norm():
    def _f(x):
        d = lambda x: torch.div(x, torch.max(x))
        return apply_batchwise(d, x)
    
    return _f


def normal(mu, sigma):
    def _f(x):
          return 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (x - mu)**2 / (2 * sigma**2))
    return _f

  
def soft_quant(classes, sigma, norm=minmax_norm(), trans=True):
    basis = torch.linspace(0, classes - 1, classes)
        
    def _f(x):
        N = normal(x * (classes - 1), sigma)
        vector = torch.stack([
            N(b) for b in basis
        ], dim=0)
        
        if norm:
            vector = norm(torch.transpose(vector, 0, 1))
            vector = torch.transpose(vector, 0, 1)
        
        if trans:
            vector = torch.transpose(vector, 0, 1)

        return vector
    return _f


def soft_torch(**kwargs):
    s = soft_quant(**kwargs)

    def _f(x):  # x ~ batch x elements
        return apply_batchwise(s, x)

    return _f


te = soft_torch(classes=3, sigma=1, trans=False)(idx)
print(te.shape, te)

gc = nn.Conv1d(3, 256, kernel_size=1, stride=1, bias=False)
gc(te).shape

torch.Size([4, 3, 11]) tensor([[[1.0000, 0.9067, 0.1870, 1.0000, 0.4365, 0.5388, 1.0000, 0.1947,
          0.5917, 1.0000, 0.1421],
         [0.7780, 1.0000, 0.7130, 0.8128, 1.0000, 1.0000, 0.8128, 0.7275,
          1.0000, 0.9735, 0.6216],
         [0.2227, 0.4057, 1.0000, 0.2430, 0.8428, 0.6827, 0.2430, 1.0000,
          0.6217, 0.3486, 1.0000]],

        [[0.6907, 0.3554, 0.1691, 0.5126, 0.1400, 1.0000, 0.4900, 1.0000,
          1.0000, 0.6359, 1.0000],
         [1.0000, 0.9829, 0.6779, 1.0000, 0.6169, 0.7146, 1.0000, 0.8565,
          0.6774, 1.0000, 0.6524],
         [0.5326, 1.0000, 1.0000, 0.7177, 1.0000, 0.1879, 0.7508, 0.2698,
          0.1688, 0.5785, 0.1566]],

        [[1.0000, 1.0000, 0.6171, 0.3263, 0.9488, 0.1444, 0.1453, 0.4485,
          0.7538, 0.3871, 0.8208],
         [0.7616, 0.6871, 1.0000, 0.9418, 1.0000, 0.6265, 0.6285, 1.0000,
          1.0000, 1.0000, 1.0000],
         [0.2134, 0.1737, 0.5961, 1.0000, 0.3877, 1.0000, 1.0000, 0.8203,
          0.4881, 0.9504, 0

torch.Size([4, 256, 11])

In [3]:
def get_figsize(n_rows, n_cols):
    row_size = 8  # heigth
    column_size = 20  # width

    return (n_cols * column_size, n_rows * row_size)


def get_figa(n_rows, n_cols):
    fig, ax = plt.subplots(n_rows, n_cols, figsize=get_figsize(n_rows, n_cols))
    return fig, ax

In [133]:
n_soft_classes = 256


def normal(mu, sigma):
    def _f(x):
          return 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (x - mu)**2 / (2 * sigma**2))
    return _f

  
def soft_quant(classes=32, sigma=10, norm=None):
    def _f(x):
        N = normal(x, sigma)
        vector = [
            N(b)
            for b in range(classes)
        ]
        
        if norm:
            return norm(vector)
        
        return vector
    return _f
  

s = soft_quant(n_soft_classes, sigma=10, norm=softmax)
basis = list(range(n_soft_classes))

fig, axis = get_figa(1, 1)
batch = np.random.uniform(0, n_soft_classes, size=4)
representation = s(batch).T  # 4 x 256

for x, r in zip(batch, representation):    
    axis.plot(basis, r, label='{:.3f}'.format(x))
    axis.vlines(x=x, ymin=min(r), ymax=max(r), color='black', linestyle='dotted')
    
axis.legend()

FileNotFoundError: [Errno 2] No such file or directory: '/home/stefano/scratch/_kernels/.venv/lib/python3.8/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf'

<Figure size 1440x576 with 1 Axes>

In [5]:
softmax([0.5, 0.2])

array([0.57444252, 0.42555748])